# QARP Model

### 1. Compute Factors

In [1]:
# import libraries
import pandas
import numpy as np
import datetime
import collections

In [2]:
# helper functions

""" Gets the lagged difference for each company. """
def diffCol(col, gvkey, step):
    return stepLookback([col], gvkey, [step], lambda x: x["col0"] - x["col0_shift0"])

""" Gets a binary column of whether the lagged and current values are negative. """
def negLookback(col, gvkey, step):
    return stepLookback([col], gvkey, [step], lambda x: 1 if x["col0"] < 0 and x["col0_shift0"] < 0 else 0)

""" Gets a column of the sum of absolute values of lagged and current values. """
def absAddition(col, gvkey, step):
    return stepLookback([col], gvkey, [step], lambda x: abs(x["col0"]) + abs(x["col0_shift0"]))

""" Gets a column of covariances of the past k values between two columns. """
def covariance(col1, col2, gvkey, k):
    def covarFunc(row):
        data1 = [row["col0"]]
        data2 = [row["col1"]]
        for i in range(k):
            data1.append(row["col0_shift" + str(i)])
            data2.append(row["col1_shift" + str(i)])
        return np.cov(np.array(data1), np.array(data2))[0][1]
    return stepLookback([col1, col2], gvkey, list(range(1, k+1)), covarFunc)

""" Gets a column of variances of the past k values for a column. """
def variance(col, gvkey, k):
    def varFunc(row):
        data = [row["col0"]]
        for i in range(k):
            data.append(row["col0_shift" + str(i)])
        return np.var(np.array(data))
    return stepLookback([col], gvkey, list(range(1, k+1)), varFunc)

""" Gets a column of standard deviations of the past k values for a column. """
def stdev(col, gvkey, k):
    def varFunc(row):
        data = [row["col0"]]
        for i in range(k):
            data.append(row["col0_shift" + str(i)])
        return np.std(np.array(data))
    return stepLookback([col], gvkey, list(range(1, k+1)), varFunc)

""" Applies func() to col[i] and col[i-step] all columns given, if they have the same gvkey. """
def stepLookback(column, gvkey, step, func, output_name="diffCol"):
    dt = pandas.DataFrame()
    
    # Copy all columns to dt
    if not isinstance(column, collections.Iterable):
        column = [column]
    for i, col in enumerate(column):
        dt["col" + str(i)] = col
    
    # Add the shifted columns to dt
    if not isinstance(step, collections.Iterable):
        step = [step]
    for i, s in enumerate(step):
        for j, col in enumerate(column):
            dt["col" + str(j) + "_shift" + str(i)] = col.shift(s)
        dt["gvkeyDiff" + str(i)] = gvkey.diff(periods=s)
    
    # Call func() on all rows such that the shifted columns have the same gvkey
    dt[output_name] = dt.apply(lambda x: func(x) if all([x["gvkeyDiff" + str(i)] == 0 for i in range(len(step))]) \
                                               else float("NaN"), axis=1)

    return dt[output_name]

In [3]:
# Import csv data
compustat = pandas.read_csv("Compustat19612018.csv")
# Used dtypes here to speed up function
crsp = pandas.read_csv("CRSP1960_2018.csv", \
    dtype={'PERMNO': int, 'date': str, 'EXCHCD': float, 'SICCD': str, 'SHRCLS': str, 'PRIMEXCH': str, \
           'PRC': float, 'RET': str, 'SHROUT': float, 'CFACSHR': float, 'vwretd': float, 'sprtrn': float})
cpi = pandas.read_csv("CPIData.csv", dtype={'Year': int, 'Annual': float})
cpi.set_index('Year', inplace=True)
linking_table = pandas.read_csv('CompustatMergedDatabase.csv')

#### Link CRSP and Compustat

In [4]:
# Fix LINKDT and LINKENDDT dates
linking_table.LINKDT = pandas.to_datetime(linking_table.LINKDT.astype(str), format='%Y%m%d', errors='coerce').fillna(datetime.date.today())
linking_table.LINKENDDT = pandas.to_datetime(linking_table.LINKENDDT.astype(str), format='%Y%m%d', errors='coerce').fillna(datetime.date.today())

# Merge compustat and linking_table based on gvkey to get permnos for each compustat company
compustat_link = compustat.merge(linking_table, on='gvkey')

# Drop dates that are not in the effective range in linking_table
compustat_link.datadate = pandas.to_datetime(compustat_link.datadate.astype(str), format='%m/%d/%Y')
compustat_link = compustat_link[(compustat_link.datadate >= compustat_link.LINKDT) & (compustat_link.datadate <= compustat_link.LINKENDDT)]

# Drop data that will cause duplication
compustat_link = compustat_link[(compustat_link.indfmt == 'INDL') & (compustat_link.consol == 'C') & (compustat_link.popsrc == 'D') & (compustat_link.datafmt == 'STD')]

# Create year column in crsp to link with permno
crsp['year'] = pandas.to_datetime(crsp.date, format='%m/%d/%Y').dt.year;

# Create permnoyear column for crsp and compustat
crsp['permnoyear'] = crsp.PERMNO.map(str) + crsp.year.map(str)
compustat_link['permnoyear'] = compustat_link.LPERMNO.map(str) + compustat_link.fyear.map(str)

# Drop the permnoyear that are duplicated due to companies' mergings
compustat_link = compustat_link.sort_values(by=['permnoyear','datadate']).drop_duplicates(subset=['permnoyear'],keep='last')

# For each year, we need the most recent value, so I am goruping it by permnoyear and then
# just taking the tail (last member) of the group. Its quick and dirty and we probably want 
# a better way to do this so we know we are getting the last value if the data isn't sorted
crspGetLastYear = crsp[['permnoyear','EXCHCD','SICCD','SHROUT' ,'SHRCLS','PRIMEXCH', 'date']]
crspGetLastYear = crspGetLastYear.groupby('permnoyear').tail(1)

# These columns need to be summed for each year, which is being done here
# RET is a str column and sometimes has error characters so still need to handle that
crspSum = crsp[['permnoyear', 'PRC', 'RET', 'vwretd', 'sprtrn']]
crspSum = crspSum.groupby(by=['permnoyear'])['sprtrn', 'PRC', 'RET', 'vwretd', 'sprtrn'].sum()

# Merge the two annualized subset dataframes back together
crsp_link = crspSum.merge(crspGetLastYear, on='permnoyear')

# Merge crsp and compustat by permnoyear and convert permnoyear to ints
model_data = compustat_link.merge(crsp_link, on='permnoyear')
model_data["permnoyear"] = pandas.to_numeric(model_data["permnoyear"])

# Sort the merge table by permnoyear
model_data = model_data.sort_values(by=["permnoyear"])

In [5]:
# Compute CPI
maxYear = cpi.index[-1]
model_data['cpi'] = model_data.apply(lambda x: cpi.loc[x['fyear']] if x['fyear'] <= maxYear else float("nan"), axis=1)

# A frame to store all the factors that are computed
factor_data = model_data["SICCD"].to_frame()

#### Profitability Factors
1. gross profits over assets (GPOA) = (Revenue - costs of goods sold) / total assets
2. return on equity (ROE) = net income / book-equity 
3. return on assets (ROA) = net income / total assets
4. cash flow over assets (CFOA) = (net income + depreciation - changes in working capital - capital expenditures) / total assets
5. gross margin (GMAR) = (revenue - cost of goods sold) / total sales
6. low accruals (ACC) = - (change in working capital - depreciation) / total assets


In [6]:
# GPOA
gp = model_data.revt - model_data.cogs
factor_data["gpoa"] = gp / model_data["at"]

# ROE
be = (model_data["seq"] - model_data["pstk"]).fillna(model_data["ceq"] + model_data["pstk"]).fillna(model_data["at"] - model_data["lt"] + compustat["mibt"])
factor_data["roe"] = model_data["ib"] / be

# ROA
factor_data["roa"] = model_data["ib"] / model_data["at"]

# CFOA
wc = model_data["act"] - model_data["lct"] - model_data["che"] + model_data["dlc"] + model_data["txp"]
wcDiff = diffCol(wc, model_data["gvkey"], 1)
cf = model_data["ib"] + model_data["dp"] - wcDiff - model_data["capx"]
factor_data["cfoa"] = cf / model_data["at"]

# GMAR
factor_data["gmar"] = gp / model_data["sale"]

# ACC
factor_data["acc"] = -wcDiff / model_data["at"]

#### Growth Factors
Five year growth of profitability factors

In [7]:
# Store for calculating later
ib = model_data["ib"]
diffIb = diffCol(ib, model_data["gvkey"], 5)
diffGp = diffCol(gp, model_data["gvkey"], 5)
atShift = model_data["at"].shift(5)
ceqShift = model_data["ceq"].shift(5)

# Growth factors
factor_data["delGpoa"] = diffGp / atShift
factor_data["delRoe"] = diffIb / ceqShift
factor_data["delRoa"] = diffIb / atShift
factor_data["delCfoa"] = diffCol(cf, model_data["gvkey"], 5) / ceqShift
factor_data["delGpoa"] = diffGp / ceqShift

#### Safety Factors

1. low beta (BAB) = cov(value-weighted return, S&P 500 return) / var(S&P 500 return)
2. low leverage (LEV) = - (total debt) / total assets
3. Ohlson’s O-score (OSCORE) = - (-1.32 - 0.407 * log(ADJASSET/CPI) + 6.03 * TLTA - 1.43 * WCTA + 0.076 * CLCA - 1.72 * OENEG - 2.37 * NITA - 1.83 * FUTL + 0.285 * INTWO - 0.521 * CHIN)
    *  Adjusted Total Assets (ADJASSET) = total assets + 10% * (Market equity - Book Equity)
    * Consumer Price Index (CPI)
    * Book Value of Debt (TLTA) = book value of debt / ADJASSET
    * Working Capital to Assets (WCTA) = (current assets - current liabilities) / ADJASSET
    * Current Liabilities to Assets (CLCA) = current liabilities / current assets
    * OENEG = 1 if total liabilities exceed total assets
    * Net income to asssets (NITA) = net income / total assets
    * Pretax Income to Liabilities(FUTL) = pretax income / total liabilites
    * INTWO = 1 if net income is negative for the current and prior fiscal year
    * (CHIN) = change in net income
4. Altmans Z-Score (AZSCORE) = (1.2 Working Capital + 1.4 Retained Earnings + 3.3 EBIT + 0.6 Market Cap + Sales) / Total Assets
5. low ROE volatility (EVOL) = Standard deviation of quarterly ROE over the past 60 quarters or 5 years (if quarterly null)


In [8]:
# Safety factors

# BAB
factor_data["bab"] = covariance(model_data["vwretd"], model_data["sprtrn"], model_data["gvkey"], 5) \
        / variance(model_data["sprtrn"], model_data["gvkey"], 5)

# LEV
factor_data["lev"] = -(model_data["dltt"] + model_data["dlc"] + model_data["mibt"] + model_data["pstk"]) / model_data["at"]

# EVOL
factor_data["evol"] = stdev(factor_data["roe"], model_data["gvkey"], 15)

# Oholn's O-score
adjasset = model_data["at"] + 0.1*(model_data["csho"] * model_data["PRC"] - model_data["ceq"])
tlta = (model_data["dlc"] + model_data["dltt"]) / adjasset
wcta = (model_data["act"] - model_data["lct"]) / adjasset
clca = model_data["lct"] / model_data["act"]
oeneg = model_data["lt"] - model_data["at"]
oeneg[oeneg <= 0] = 0
nita = model_data["ib"] / model_data["at"]
futl = model_data["pi"] / model_data["lt"]
intwo = negLookback(model_data["ib"], model_data["gvkey"], 1)
chin = diffCol(model_data["ib"], model_data["gvkey"], 1) / absAddition(model_data["ib"], model_data["gvkey"], 1)
factor_data["oscore"] = -(-1.32 - 0.407 * np.log(adjasset / model_data["cpi"]) + 6.03 * tlta - 1.43 * wcta \
            + 0.076 * clca - 1.72 * oeneg - 2.37 * nita - 1.83 * futl + 0.285 * intwo - 0.521 * chin)

# Altman's Z-score
ebit = model_data["ib"] + model_data["tie"] + model_data["txt"]
me = model_data["csho"] * model_data["PRC"]
factor_data["azscore"] = (1.2 * wc + 1.4 * model_data["re"] + 3.3 * ebit + 0.6 * me) / model_data["at"]

ValueError: Wrong number of items passed 2, placement implies 1

In [ ]:
# Payout factors

# EISS
shrout_adj = model_data['SHROUT'] * model_data['CFACSHR']
factor_data["eiss"] = -np.log(diffCol(model_data['SHROUT'], model_data["gvkey"], 1))

# DISS
totd = model_data['dltt'] + model_data['dlc'] + model_data['mibt'] + model_data['pstk']
factor_data["diss"] = -np.log(diffCol(totd, model_data["gvkey"], 1))

# NPOP
factor_data["npop"] = (model_data['ib'] - diffCol(be, model_data["gvkey"], 1)) / (model_data['revt'] - model_data['cogs'])

"""
#get z-scores per sector
#calc means and stdevs for each sector
#within each sector, calc z-scores for each company

sic_codes = {"Agriculture, Forestry and Fishing":(0100,0999), #need to fix
             "Mining":(1000,1499),
             "Construction":,
             "Manufacturing":,
             "Transportation":,
             "Wholesale Trade":,
             "Retail Trade":,
             "Finance, Insurance and Real Estate":,
             "Services":,
             "Public Administration":,
             "Nonclassifiable":,
            } #dictionary of sector name to tuple of start-stop SIC range

prof_means = {}
grow_means = {}
safe_means = {}
pay_means = {}

std_devs = {}

for sector in sic_codes:
    df = model_data[(model_data['SICCD'] >= sic_codes[sector][0]) & (model_data['SICCD'] <= sic_codes[sector][1])]
    prof_means[sector] = df[['gpoa', 'roe', 'roa', 'cfoa', 'gmar', 'acc']].mean() #this way copies columns, may be slower
    grow_means[sector] = df[['delGpoa', 'delRoe', 'delRoa', 'delCfoa', 'delGpoa']].mean()
    safe_means[sector] = df[['bab', 'lev', 'oscore', 'azscore', 'evol']].mean()
    pay_means[sector] = df[['eiss', 'diss', 'npop']].mean()
    #same with st_devs
    """


In [ ]:
# Calculate z-scores
model_data["industry"] = model_data["SICCD"].apply(lambda x: print("y", x) if float(x) != float("nan") else print("n", x))
factor_data["industry"] = model_data["industry"]
grouped = model_data.groupby('industry')
means = grouped.apply(lambda x: x.mean())
stdDevs = grouped.std()

def zScore(row):
    newRow = []
    for column in row.columns:
        newRow[column] = (row[column] - means.loc[row["industry"]][column]) \
                            / stdDevs.loc[row["industry"]][column]
    return newRow

z_scoreFactor = factor_data.apply(lambda x: zScore(x), axis=1)

In [ ]:
print(z_scoreFactor)